In [11]:
import sys; sys.path.insert(0,'../')
from dist_tools import *

import pandas as pd
import geopandas as gpd
from IPython.display import display

import psycopg2
from netrc import netrc
from fiona.crs import from_epsg

user, acct, passwd = netrc().authenticators("harris")

seats, fips, epsg = 8, 27, 3594

%matplotlib inline
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 10000

/home/jsaxon/anaconda3/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [27]:
mnf = "/media/jsaxon/brobdingnag/data/el_dv/mn_site/"
map_08 = gpd.read_file(mnf + "elec2008.shp")
map_08.rename(columns = {"USPRESDFL" : "D08", "USPRESR" : "R08"}, inplace = True)
map_08 = map_08[["D08", "R08", "geometry"]]
map_08["DFrac"] = map_08.D08 / (map_08.D08 + map_08.R08)
map_08 = map_08.to_crs(epsg = epsg)

map_12 = gpd.read_file(mnf + "elec2012.shp")
map_12.rename(columns = {"USPRSDFL" : "D12", "USPRSR" : "R12"}, inplace = True)
map_12 = map_12[["D12", "R12", "geometry"]]
map_12["DFrac"] = map_12.D12 / (map_12.D12 + map_12.R12)
map_12 = map_12.to_crs(epsg = epsg)

map_16 = gpd.read_file(mnf + "elec2016.shp")
map_16.rename(columns = {"USPRSDFL" : "D16", "USPRSR" : "R16"}, inplace = True)
map_16 = map_16[["D16", "R16", "geometry"]]
map_16["DFrac"] = map_16.D16 / (map_16.D16 + map_16.R16)
map_16 = map_16.to_crs(epsg = epsg)

In [28]:
output_geojson(map_08.set_geometry(map_08.geometry.simplify(10)), var = "DFrac", fname = "mapped/mn_2008.geojson")
output_geojson(map_12.set_geometry(map_12.geometry.simplify(10)), var = "DFrac", fname = "mapped/mn_2012.geojson")
output_geojson(map_16.set_geometry(map_16.geometry.simplify(10)), var = "DFrac", fname = "mapped/mn_2016.geojson")

In [5]:
query = """SELECT 
              rn, ST_Transform(tr.geom, epsg) geometry
           FROM census_tracts_2015 AS tr
           JOIN (SELECT state, county, tract,
                        row_number() over 
                          (PARTITION BY state ORDER BY county, tract NULLS LAST) - 1 as rn
                 FROM census_tracts_2015) rn ON
                   tr.state  = rn.state  AND
                   tr.county = rn.county AND
                   tr.tract  = rn.tract
           JOIN states AS st ON st.fips = tr.state
           WHERE tr.state = {} ORDER BY rn;
           """

con = psycopg2.connect(database = "census", user = user, password = passwd,
                       host = "saxon.harris.uchicago.edu", port = 5432)

tr_rn = gpd.GeoDataFrame.from_postgis(query.format(fips), con, geom_col = "geometry", crs = from_epsg(epsg))

In [6]:
tr_rn.head()

,rn,geometry
0,0,(POLYGON ((841559.2706760777 275179.5267206162...
1,1,(POLYGON ((836121.8174649179 281250.5153568669...
2,2,(POLYGON ((833603.7923484842 267296.7002479853...
3,3,(POLYGON ((859078.8538517348 267617.4044722447...
4,4,(POLYGON ((853172.4178123451 248488.2945790889...


In [22]:
votes08 = merge_tract_number(tr_rn, map_08)[["D08", "R08"]]
votes12 = merge_tract_number(tr_rn, map_12)[["D12", "R12"]]
votes16 = merge_tract_number(tr_rn, map_16)[["D16", "R16"]]

In [24]:
pd.concat([votes08, votes12, votes16], axis = 1).to_csv("votes/mn.csv")

In [25]:
pd.read_csv("votes/mn.csv").head()

,rn,D08,R08,D12,R12,D16,R16
0,0,692.0,769.0,651.0,789.0,474.0,913.0
1,1,624.0,539.0,614.0,515.0,402.0,746.0
2,2,706.0,793.0,668.0,777.0,512.0,908.0
3,3,937.0,839.0,924.0,796.0,644.0,975.0
4,4,625.0,592.0,534.0,580.0,334.0,686.0
